### If Execute code on Colab

In [ ]:
# # Google Drive mount
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !pip install huggingface_hub transformers bitsandbytes datasets
# !pip install -U transformers accelerate bitsandbytes torch torchvision torchaudio
# !pip install --upgrade datasets

In [ ]:
# from datasets import load_dataset, DatasetDict

# # Load dataset
# dataset = load_dataset("pubmed_qa", "pqa_labeled")

# # Split
# train_valid_test = dataset["train"].train_test_split(test_size=0.2, seed=42)
# dev_test = train_valid_test["test"].train_test_split(test_size=0.5, seed=42)

# # Group into DatasetDict
# dataset_splits = DatasetDict({
#     "train": train_valid_test["train"],
#     "dev": dev_test["train"],
#     "test": dev_test["test"]
# })

# # Save to disk
# dataset_splits.save_to_disk("data")

### IA3

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import get_peft_model, IA3Config, TaskType
from datasets import load_from_disk

In [ ]:
# 1. Load tokenizer and model
# model name
model_id = "Qwen/Qwen3-1.7B"

# IA3
peft_config = IA3Config(
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    feedforward_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    task_type="CAUSAL_LM"
)

# model with IA3
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# 2. Load dataset (from disk)
from datasets import DatasetDict

dataset = DatasetDict.load_from_disk("/content/drive/MyDrive/LLM/few-shot/data")

train_dataset = dataset["train"]
dev_dataset = dataset["dev"]
test_dataset = dataset["test"]

print(train_dataset[0])

In [ ]:
# 3. Preprocessing: format prompt and tokenize
def format_pubmedqa(example):
    prompt = f"Question: {example['question']}\nContext: {example['context']}\nAnswer:"
    target = example["long_answer"]
    full_text = prompt + " " + target

    tokenized = tokenizer(
        full_text,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    tokenized["labels"] = tokenized["input_ids"].clone()  # 수정된 부분
    return {k: v.squeeze() for k, v in tokenized.items()}

tokenized_train = train_dataset.map(format_pubmedqa, remove_columns=train_dataset.column_names)

In [ ]:
# 4. Training arguments
training_args = TrainingArguments(
    output_dir="./results/qwen_pubmedqa_ia3_sft",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    logging_dir="./logs",
    logging_steps=20,
    save_strategy="epoch",
    save_total_limit=1,
    fp16=True,
    report_to="none"
)

# 5. Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

# 6. Start training
trainer.train()

In [ ]:
# 7. Save model and tokenizer
model.save_pretrained("./results/qwen_pubmedqa_ia3_sft/final")
tokenizer.save_pretrained("./results/qwen_pubmedqa_ia3_sft/final")

In [ ]:
# 8. Save model to huggingface
from huggingface_hub import login, create_repo
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["HUGGINGFACE_TOKEN"] = os.getenv("HUGGINGFACE_TOKEN")

# Huggingface Login
login(token = os.environ["HUGGINGFACE_TOKEN"])

# repository
repo_id = "[your_huggingface_id]/qwen_pubmedqa_ia3_sft"
create_repo(repo_id, private=False)

# model & tokenizer load
model_path = "./results/qwen_pubmedqa_ia3_sft/final"
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Huggingface Upload
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

### Model Test

In [ ]:
# Model Test
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = '[your_huggingface_id]/qwen_pubmedqa_ia3_sft'

model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
prompt = "Aspirin reduces the risk of heart attack?"
inputs = tokenizer(prompt, return_tensors="pt")
output = model.generate(**inputs, max_length=200, do_sample=True, top_k=50, top_p=0.95)
decode = tokenizer.decode(output[0], skip_special_tokens=True)

print("Answer:")
print(decode)